<h1>Лабораторна робота #3 </h1>
<h2>ФБ-33 Грабченко Олександр</h2>



<p>
  3. Створіть веб додаток за допомогою бібліотеки Spyre
  (https://dataspyre.readthedocs.io/en/latest/)
</p>
<p>
  4. Додати випадаючий список drop-down list як наведено в example 2
  (https://dataspyre.readthedocs.io/en/latest/getting_started.html#example-2-tabs-and-tables)
  Випадаючий список має містити VCI, TCI, VHI
  Де значення беруться із датафрему df з минулої лабораторної роботи (df[[‘VCI’,’TCI’,’VHI’]]).
  Не забуваємо, що key це параметр за допомогою якого ви можете отримувати вибране
  значення. Логічні операції вам в допомогу.
</p>
<p>
  5. Створити ще один випадаючий список для вибору області
</p>
<p>
  6. Створити текстове поле для введення інтервалу місяців
</p>
<p>
 7. Створити таблицю та графік для відображення даних
</p>

In [1]:
from spyre import server
import __main__
__main__.__file__ = "main_file"

import pandas
from urllib.request import urlopen, Request
from urllib.error import HTTPError
import json
import datetime
import os
import re

In [2]:
province_path = "provinces.json"
country_path = "countries.json"
province_order_path = 'province_changed_order.json'

def get_sp_provinces(country):
    with open(province_path) as fd:
        province_json:dict = json.load(fd)
    
    country_provinces:dict = province_json[country]
    sp_provinces = []

    for province_id, province_name in country_provinces.items():
        sp_provinces.append({"label": province_name, "value": province_id})

    province_number = len(sp_provinces)
    return sp_provinces, province_number

In [3]:
class Data:
    def __init__(self, dir_path, country):
        _, province_number = get_sp_provinces(country)
        self.df = None
        self.country = country
        self.dir_path = os.path.join(dir_path, country)
        self.province_number = province_number

    def formatData(self, unformatted_data):
        formatted_data = re.sub("<[^<]+?>", '', unformatted_data)
        return formatted_data
    

    def clearDir(self):
        for file_name in os.listdir(self.dir_path):
            file_path = os.path.join(self.dir_path, file_name)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print("Failed to delete {}. Reason: {}".format(file_path, e))


    def downloadDataForProvince(self, province_id, start_year=1981, end_year=None):
        try:
            curr_year = datetime.datetime.now().year

            if (end_year is None) or (end_year > curr_year) or (end_year < start_year):
                end_year = curr_year

            data_type = "Mean" 
            curr_datetime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

            data_url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php" + \
                "?country={}&provinceID={}&year1={}&year2={}&type={}".format(self.country.upper(), province_id, str(start_year), str(end_year), data_type)

            data_path = "{}/NOAA_ID{}_{}.csv".format(self.dir_path, province_id, curr_datetime)

            request = Request(data_url, method="GET")
            responce = urlopen(request)
            raw_data = responce.read()
            unformatted_data = raw_data.decode('utf-8')

            formated_data = self.formatData(unformatted_data)
            
            with open(data_path, "w") as raw_data_file:
                raw_data_file.write(formated_data)

        except HTTPError as e:
            print(e.code())
            print(e.read())


    def downloadData(self):
        if not (os.path.exists(self.dir_path) and os.path.isdir(self.dir_path)):
            print("{} does not exist. Creating...")
            os.makedirs(self.dir_path, exist_ok=True)
            print("Data does not exist. The download will still happen.")
        else:
            self.clearDir()

        for i in range(1, self.province_number + 1):
            self.downloadDataForProvince(i) 
    

    def getProvinceName(self, file_path):
        data_file =  open(file_path, 'r')
        data_info = data_file.readline()

        province_name = data_info[data_info.find(":") + 2:data_info.find(",")]
        province_id = data_info[data_info.find("=") + 2:data_info.find(":")]
        data_file.close()
        return province_id, province_name


    def createDataframeForProvince(self, file_path):
        headers = ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI", "empty"]

        province_id, _ = self.getProvinceName(file_path)
        df = pandas.read_csv(file_path, header=1, names=headers)
        df.insert(0, "province_id", province_id, True)
        df = df.drop(columns=["empty"])
        return df

    def createDataframe(self):
        if not (os.path.exists(self.dir_path) and os.path.isdir(self.dir_path)):
            print("{} does not exists.".format(self.dir_path))
            return

        province_df_list = []
        for file_name in os.listdir(self.dir_path):
            file_path = os.path.join(self.dir_path, file_name)
            province_df = self.createDataframeForProvince(file_path)
            province_df_list.append(province_df)

        country_df = pandas.concat(province_df_list)
        self.df = country_df

    def changeProvinceIndeces(self):
        with open(province_order_path, 'r') as province_order_json:
            province_order:dict = json.load(province_order_json)

        country_province_order:dict = province_order[self.country]
        for old, new in country_province_order.items():
            self.df.loc[self.df["province_id"] == old, "province_id"] = int(new)
    

    def clearDataFrame(self):
        self.df = self.df.drop(self.df.loc[(self.df["VHI"] == -1)].index)
    
    def getDataFrame(self):
        if self.df is None:
            self.createDataframe()
        
        return self.df


In [4]:
class WebApp(server.App):

    def __init__(self, df, country):
        super().__init__()
        sp_provinces, _ = get_sp_provinces(country)
        min_year = 1982
        max_year = datetime.datetime.now().year

        self.title = "NOAA Data Visualisation"
        self.df = df
        self.country = country

        self.inputs = [
            dict(
                type="dropdown",
                label= "Indices types",
                options=[
                    {"label": "VCI", "value": "VCI"},
                    {"label": "TCI", "value": "TCI"},
                    {"label": "VHI", "value": "VHI"},
                ],
                key="indices_type",
                action_id="update_data",
            ),
            dict(
                type="dropdown",
                label="Province",
                options=sp_provinces,
                key="province",
                action_id="update_data",
            ),
            dict(
                type="slider",
                label="Year",
                key="year",
                value=2000,
                min=min_year,
                max=max_year,
                action_id="update_data",
            ),
            dict(
                type="text",
                key="week_range",
                label="Week range (1/52)",
                value="2/10",
                action_id="update_data",
            ),
        ]

        self.controls = [{
            "type" : "hidden",
            "id" : "update_data"
        }]

        self.tabs = [
            "Plot",
            "Table",
        ]

        self.outputs = [
            dict(
                type="plot",
                id="get_table",
                control_id="update_data",
                tab="Plot",
            ),
            dict(
                type="table",
                id="table",
                control_id="update_data",
                tab="Table",
                on_page_load=True,
            ),
        ]


    def getProvinceName(self, province_id):
        with open(province_path) as fd:
            provinces_dict = json.load(fd)

        return provinces_dict[self.country][str(province_id)]

    def getCountryName(self):
        with open(country_path) as fd:
            countries_dict = json.load(fd)
        
        return countries_dict[self.country]

    def prepareData(self, params):
        year = params["year"]
        
        week_range:list[str] = params["week_range"].split('/')
        if len(week_range) != 2:
            return
        week_low = 0
        week_high = 0
        if not (week_range[0].isnumeric() and week_range[1].isnumeric()):
            return
        else:
            week_low = int(week_range[0])
            week_high = int(week_range[1])
        
        province_id = int(params["province"])

        df:pandas.DataFrame = self.df.loc[
            (self.df["province_id"] == province_id)
            & (self.df["year"] == year)
            & (self.df["week"] >= week_low)
            & (self.df["week"] <= week_high)
        ]

        return df

    def getPlot(self, params):
        indices_type = params["indices_type"]
        year = int(params["year"])
        df:pandas.DataFrame = self.prepareData(params)[["week", indices_type]].set_index("week")

        province_id = int(params["province"])
        province_name = self.getProvinceName(province_id)
        font = {'family':'serif','color':'black','size':12}
        country = self.getCountryName()
        plt_obj = df.plot(color='r', linewidth='1.0', marker='o', mfc='k', mec='k', ms=2)
        
        plt_obj.set_ylabel(indices_type, fontdict=font)
        plt_obj.set_xlabel("Weeks", fontdict=font)
        plt_obj.set_title("Dependence of {} on time ({}, {}, {})".format(indices_type, country, province_name, year), fontdict=font)
        plt_obj.grid(visible=True, color='blue', linestyle='--', linewidth='0.4')

        fig = plt_obj.get_figure()
        fig.set_figheight(7)
        fig.set_figwidth(16)
        return fig

    def getTable(self, params):
        df = self.prepareData(params)[["province_id", "year", "week", params["indices_type"]]]
        return df

In [5]:
country = "ukr"
data_dir_path = "dfs"


data = Data(country=country, dir_path=data_dir_path)
data.downloadData()
data.createDataframe()
data.changeProvinceIndeces()
data.clearDataFrame()

app = WebApp(data.getDataFrame(), country)

{} does not exist. Creating...
Data does not exist. The download will still happen.


In [ ]:
app.launch()

[17/Apr/2025:12:23:52] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Listening for SIGTERM.
[17/Apr/2025:12:23:52] ENGINE Bus STARTING
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Bus STARTING
[17/Apr/2025:12:23:52] ENGINE Set handler for console events.
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[17/Apr/2025:12:23:52] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Started monitor thread 'Autoreloader'.
[17/Apr/2025:12:23:52] ENGINE Serving on http://127.0.0.1:8080
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Serving on http://127.0.0.1:8080
[17/Apr/2025:12:23:52] ENGINE Bus STARTED
INFO:cherrypy.error:[17/Apr/2025:12:23:52] ENGINE Bus STARTED


127.0.0.1 - - [17/Apr/2025:12:25:27] "GET / HTTP/1.1" 200 442868 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


INFO:cherrypy.access.2362400324688:127.0.0.1 - - [17/Apr/2025:12:25:27] "GET / HTTP/1.1" 200 442868 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


INFO:cherrypy.access.2362400324688:127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


...

INFO:cherrypy.access.2362400324688:127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /table?indices_type=VCI&province=1&year=__float__2000&week_range=2/10&output_id=table& HTTP/1.1" 200 1230 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
ERROR:root:Error: getPlot method must return an pyplot figure or matplotlib Axes object


127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /plot?indices_type=VCI&province=1&year=__float__2000&week_range=2/10&output_id=get_table& HTTP/1.1" 200 44472 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


INFO:cherrypy.access.2362400324688:127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /plot?indices_type=VCI&province=1&year=__float__2000&week_range=2/10&output_id=get_table& HTTP/1.1" 200 44472 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


INFO:cherrypy.access.2362400324688:127.0.0.1 - - [17/Apr/2025:12:25:27] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"


<p>
  Приклад роботи:
</p>
<p>
   - Графік
</p>
<img src="images/Screenshot 2024-03-29 010443.png">
<p>
  - Таблиця
</p>
<img src="images/Screenshot 2024-03-29 010512.png">